In [2]:
import pandas as pd
import configparser
import os

# Read the value from the config.ini file
config = configparser.ConfigParser()
config.read('config.ini')
output_file_dir = config.get('path', 'output_file_dir')
output_file_name_net_sales = config.get('path', 'output_file_name_net_sales')
# using os.path.join() method
# output_file_dir = os.path.join(output_file_dir, 'outputFile')
net_sales_data = os.path.join(output_file_dir, output_file_name_net_sales) + '.csv'
# Read the csv file
# input_file should be the path to the merge csv file
try:
    input_file = net_sales_data
    df = pd.read_csv(input_file)
except FileNotFoundError as e:
    print(e)
    # input_file = r"C:\Users\Trieu Pham\OneDrive - BTM Global Consulting\Projects\data csv\outputFile\data.csv"
    # df = pd.read_csv(input_file)
df.dropna(inplace=True)

In [3]:
df

,Date,Time,InvoiceID,Barcode,Total Include VAT
0,2023-01-01,00:40:59,VN0001010101230001,8935049500544,7000
1,2023-01-01,00:41:18,VN0001010101230002,8938512632025,12000
2,2023-01-01,00:41:55,VN0001010101230003,8936011773416,25000
3,2023-01-01,00:41:55,VN0001010101230003,8936079121761,13000
4,2023-01-01,00:41:55,VN0001010101230003,8850453017528,13000
...,...,...,...,...,...
2054556,2023-01-11,22:06:27,VN0236021101230057,8935112200005,13000
2054557,2023-01-11,22:10:29,VN0236021101230058,8934803044027,13000
2054558,2023-01-11,22:10:29,VN0236021101230058,8935001215028,11000
2054559,2023-01-11,18:19:22,VN9996011101230005,2010101000012,9000


In [5]:
# Add new column StoreID (first 6 digits of InvoiceID)
df['StoreID'] = df['InvoiceID'].str[:6]

# Add new column WeekDay (1: Sunday, 2: Monday, ..., 7: Saturday)
df['WeekDay'] = pd.to_datetime(df['Date']).dt.weekday + 2
df.loc[df['WeekDay'] > 7, 'WeekDay'] = 1

df

,Date,Time,InvoiceID,Barcode,Total Include VAT,StoreID,WeekDay
0,2023-01-01,00:40:59,VN0001010101230001,8935049500544,7000,VN0001,1
1,2023-01-01,00:41:18,VN0001010101230002,8938512632025,12000,VN0001,1
2,2023-01-01,00:41:55,VN0001010101230003,8936011773416,25000,VN0001,1
3,2023-01-01,00:41:55,VN0001010101230003,8936079121761,13000,VN0001,1
4,2023-01-01,00:41:55,VN0001010101230003,8850453017528,13000,VN0001,1
...,...,...,...,...,...,...,...
2054556,2023-01-11,22:06:27,VN0236021101230057,8935112200005,13000,VN0236,4
2054557,2023-01-11,22:10:29,VN0236021101230058,8934803044027,13000,VN0236,4
2054558,2023-01-11,22:10:29,VN0236021101230058,8935001215028,11000,VN0236,4
2054559,2023-01-11,18:19:22,VN9996011101230005,2010101000012,9000,VN9996,4


In [7]:
# Backup the dataframe
df_backup = df.copy()

df_backup

,Date,Time,InvoiceID,Barcode,Total Include VAT,StoreID,WeekDay
0,2023-01-01,00:40:59,VN0001010101230001,8935049500544,7000,VN0001,1
1,2023-01-01,00:41:18,VN0001010101230002,8938512632025,12000,VN0001,1
2,2023-01-01,00:41:55,VN0001010101230003,8936011773416,25000,VN0001,1
3,2023-01-01,00:41:55,VN0001010101230003,8936079121761,13000,VN0001,1
4,2023-01-01,00:41:55,VN0001010101230003,8850453017528,13000,VN0001,1
...,...,...,...,...,...,...,...
2054556,2023-01-11,22:06:27,VN0236021101230057,8935112200005,13000,VN0236,4
2054557,2023-01-11,22:10:29,VN0236021101230058,8934803044027,13000,VN0236,4
2054558,2023-01-11,22:10:29,VN0236021101230058,8935001215028,11000,VN0236,4
2054559,2023-01-11,18:19:22,VN9996011101230005,2010101000012,9000,VN9996,4


In [14]:
# Group by InvoiceID and calculate the sum of Total Include VAT for each group
df_grouped = df.groupby('InvoiceID', as_index=False).agg({
    'StoreID': 'first',
    'Date': 'first',
    'Time': 'first',
    'Barcode': 'first',
    'WeekDay': 'first',
    'Total Include VAT': 'sum'
})

# Function to get Time Range for each Time value
def get_time_range(time_str):
    hour = int(time_str[:2])
    time_range_start = f"{hour:02}:00"
    time_range_end = f"{hour:02}:59"
    return f"{time_range_start}-{time_range_end}"

# Add new column Time Range
df_grouped['Time Range'] = df_grouped['Time'].apply(get_time_range)

df_grouped

,InvoiceID,StoreID,Date,Time,Barcode,WeekDay,Total Include VAT,Time Range
0,VN0001010101230001,VN0001,2023-01-01,00:40:59,8935049500544,1,14000,00:00-00:59
1,VN0001010101230002,VN0001,2023-01-01,00:41:18,8938512632025,1,24000,00:00-00:59
2,VN0001010101230003,VN0001,2023-01-01,00:41:55,8936011773416,1,230000,00:00-00:59
3,VN0001010101230004,VN0001,2023-01-01,00:43:12,8801100128845,1,382000,00:00-00:59
4,VN0001010101230005,VN0001,2023-01-01,00:44:22,8934588843051,1,52000,00:00-00:59
...,...,...,...,...,...,...,...,...
830120,VN0236021101230055,VN0236,2023-01-11,22:04:24,2270103000025,4,54000,22:00-22:59
830121,VN0236021101230056,VN0236,2023-01-11,22:06:01,8936127790017,4,6000,22:00-22:59
830122,VN0236021101230057,VN0236,2023-01-11,22:06:27,2010101000005,4,16000,22:00-22:59
830123,VN0236021101230058,VN0236,2023-01-11,22:10:29,8934803044027,4,24000,22:00-22:59


In [15]:
# Create a pivot table to get the matrix representation
pivot_table = df_grouped.pivot_table(index=['StoreID', 'WeekDay'], columns='Time Range', values='Total Include VAT', aggfunc='sum')
pivot_table = pivot_table.applymap("{:,.0f}".format)

pivot_table

Time Range      00:00-00:59 01:00-01:59 02:00-02:59 03:00-03:59 04:00-04:59   
StoreID WeekDay                                                               
VN0001  1         8,383,000   8,873,000   1,520,000     664,000     254,000  \
        2           862,000     758,000     439,000     510,000     280,000   
        3           833,000   1,724,000     381,000     106,000     108,000   
        4           548,000   1,431,000     225,000     282,000     190,000   
        5           416,000     143,000     224,000     114,000     278,000   
...                     ...         ...         ...         ...         ...   
VN0236  4         1,411,000     593,000     386,000     106,000     203,000   
        5           752,000   1,198,000     183,000     243,000     966,000   
        6           491,000     345,000     357,000     106,000     487,000   
        7           580,000     379,000     662,000     133,000         nan   
VN9996  4               nan         nan         nan         nan         nan   

Time Range      05:00-05:59 06:00-06:59 07:00-07:59 08:00-08:59 09:00-09:59   
StoreID WeekDay                                                               
VN0001  1           452,000     985,000   1,865,000   1,274,000   2,285,000  \
        2            34,000     672,000   2,685,000   3,779,000   3,171,000   
        3           382,000   1,230,000   4,108,000   7,012,000   3,686,000   
        4           634,000   1,351,000   5,596,000   7,583,000   5,683,000   
        5             6,000   1,064,000   2,406,000   4,623,000   2,333,000   
...                     ...         ...         ...         ...         ...   
VN0236  4           306,000   1,331,000     416,000     358,000   1,774,000   
        5           335,000     473,000     430,000     349,000     627,000   
        6           145,000     187,000     281,000     383,000     323,500   
        7            85,000     132,000     764,000     663,000     750,000   
VN9996  4               nan         nan         nan         nan         nan   

Time Range       ... 14:00-14:59 15:00-15:59 16:00-16:59 17:00-17:59   
StoreID WeekDay  ...                                                   
VN0001  1        ...   3,774,000   2,230,000   2,645,000   4,613,000  \
        2        ...   2,338,000   2,505,000   3,548,000   1,810,000   
        3        ...   2,761,000   3,990,000   4,059,000   3,247,000   
        4        ...   1,987,200   4,586,000   3,090,000   7,920,000   
        5        ...   1,825,000   1,648,000   1,745,000   2,232,000   
...              ...         ...         ...         ...         ...   
VN0236  4        ...     648,000     544,000   1,993,000     989,000   
        5        ...      37,000     327,000   1,346,000     747,000   
        6        ...     462,600     871,000   1,178,000   1,254,000   
        7        ...     549,000     936,000     742,000   1,408,000   
VN9996  4        ...         nan         nan         nan         nan   

Time Range      18:00-18:59 19:00-19:59 20:00-20:59 21:00-21:59 22:00-22:59   
StoreID WeekDay                                                               
VN0001  1         2,478,000   2,395,000   4,445,000   4,459,000   5,320,000  \
        2         2,528,000   1,726,000   1,637,000   2,489,000   1,871,000   
        3         2,947,000   2,618,000   2,658,000   2,499,000   2,379,000   
        4         2,682,300   2,520,000   2,170,000   1,668,000   2,254,000   
        5         3,396,000   1,922,000   1,196,000   1,579,000   1,624,000   
...                     ...         ...         ...         ...         ...   
VN0236  4         1,938,000   1,419,000   2,247,000   2,990,000   3,386,000   
        5           639,000     880,000     704,000   1,333,000   1,550,000   
        6           611,000   1,283,000     696,000   1,306,000   1,025,000   
        7           831,000     872,000   2,373,000   1,097,000   1,944,000   
VN9996  4            12,000         nan         nan        

In [ ]:
# Save the pivot table to csv file
# pivot_table.to_csv('result.csv')